In [23]:
#import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd

In [30]:
#define path to chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [31]:
#first url to scrape article title and content
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
title = soup.find('div', class_='content_title').get_text()
paragraph = soup.find('div', class_='article_teaser_body').get_text()
paragraph

"Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover."

In [32]:
# visit the space image website to get an image
base_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(base_url)
browser.click_link_by_partial_text('FULL')
browser.is_element_present_by_css('buttons', wait_time = 2)
browser.click_link_by_partial_text('more info')
browser.is_element_present_by_css('figure',wait_time = 2)
browser.click_link_by_partial_href('/spaceimages/images/largesize')
html = browser.html
soup = bs(html, 'html.parser')
img_src = soup.find('img').get('src')
img_src

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18087_hires.jpg'

In [140]:
#visit twitter to get weather info
base_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(base_url)
html = browser.html
soup = bs(html, 'html.parser')
tweet = soup.find_all('p',class_ = 'TweetTextSize')
 #the most recent tweet might not contain weather information
    #we want the first tweet with weather info, whichalways starts with the string 'InSight sol'
for twit in tweet:
    if 'InSight sol' in twit.text:
        pic_text = twit.a.text 
        #.text pics up additional text in an a-tag, replace keeps only the desired text
        Mars_weather = twit.text.replace(pic_text,'')
        
        break
print(Mars_weather)

InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPa
pic.twitter.com/9YLawm67zS


In [20]:
#uing pandas, the information table is obtained and put in a dataframe
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
#keep the second table
access_table = tables[1]
table_df = pd.DataFrame(access_table)
table_df = table_df.rename(columns = {0: "Description", 1:"Value"}).set_index('Description')
table_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [17]:
#the table data is then put into html code
table = table_df.to_html().replace('\n', '')
table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Description</th>      <th>Value</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronom

In [13]:
#get the hemisphere images
base_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(base_url)
html = browser.html
soup = bs(html, 'html.parser')
#get the items that identify the hemispheres
items = soup.find_all('div',class_ = 'item')
hemi_dict = []
#loop through hemispheres to get text and image
for item in items:
    h = item.find('h3').text
    #go to hew link for hemisphere
    linka = item.find('a', class_ = 'itemLink product-item')['href']
    new_url = 'https://astrogeology.usgs.gov' + linka
    browser.visit(new_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    #get image url and add to dictionary
    img_url = 'https://astrogeology.usgs.gov'+ soup.find('img', class_ = "wide-image").get('src')
    dictionary = {'title':h,'img_url':img_url}
    hemi_dict.append(dictionary)
hemi_dict

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': '/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': '/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': '/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': '/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]